# Converting LI Files and Reading `splendaq` HDF5 Files
-----
In this tutorial, we will show how to convert an LI binary file to a `splendaq` HDF5 file, and how to read the subsequent file with `splendaq`'s I/O functionality.

Let's start by creating our own example file on the Moku, using `LogData`. See the DAQ tutorials for more.

In [ ]:
from splendaq.daq import LogData

In [ ]:
moku_ip = 'your_ip_address'

with LogData(moku_ip) as LOG:
    LOG.set_input_channels(1)
    dc_settings = LOG.dc_settings(
        dc_level=0.1,
    )
    LOG.set_output_channel(1, 'DC', **dc_settings)
    LOG.log_data(10)

This creates a file with the `.li` extension (the Liquid Instruments binary format). To convert to a file format that is easy to handle, we use `splendaq.io.convert_li_to_h5` to convert the file to a custom `splendaq` HDF5 file.

Note, this currently is a wrapper of Liquid Instruments' compiled converter to a MAT file, which we then convert to an H5 file. Thus, it is not the most efficient way of doing the conversion, but the best we can do at the moment without the source code the converter in C.

In [ ]:
from splendaq.io import convert_li_to_h5

In [ ]:
convert_li_to_h5('logged_file.li', my_os='mac')

The OS must be specified in the conversion, as there are different executables for the conversion for different OSs. The supported OSs are macOS, Linux, and Windows, which can be specified with `'mac'`, `'linux'`, and `'windows'`, respectively.

We now have a `splendaq` HDF5 file, which we can read via `splendaq.io.Reader`

In [ ]:
from splendaq.io import Reader

FR = Reader('logged_file.h5')

The `Reader` class has two methods: `get_data` and `get_metadata`. If the user wants to load just the data or both the data and the metadata simultanetously, then `get_data` should be used.

In [ ]:
data = FR.get_data()
data, metadata = FR.get_data(include_metadata=True)

If the user only needs the metadata, then `get_metadata` should be used, as it does not load all of the data into memory, just the metadata.

In [ ]:
metadata = FR.get_metadata()

For the data itself, the shape is always (number of traces, number of channels, length of traces in bins). Thus, to plot just the first trace from the first channel, one would use the below code.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(data[0, 0])

Note that the active file can be changed without reinitalizing the class, such as below if we wanted the data from a separate file. Any subsequent calls to the class will now be to this new file.

In [ ]:
different_data = FR.get_data('different_dataset.h5')